In [ ]:
from neo4j import GraphDatabase
import nxneo4j as nxn
import json
import re
import networkx as nx
from networkx.readwrite import json_graph
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import ipywidgets as widgets
import holoviews as hv
hv.extension('bokeh')
from holoviews import opts
import numpy as np
import seaborn as sns

In [ ]:
uri = ""
user = "neo4j"
password = ""
driver = GraphDatabase.driver(uri, auth=(user, password))

In [ ]:
G = nxn.Graph(driver) # or nx.DiGraph(driver) for directed graphs

In [ ]:
with open("./MC3_schema.json") as f:
    schema = json.load(f)

with open("./MC3_graph.json") as g:
    graph_data = json.load(g)

G_nx = json_graph.node_link_graph(graph_data)

In [ ]:
def format_dict(d):
    parts = []
    for k, v in d.items():
        key_str = str(k)
        if v is None:
            v = " "
        if isinstance(v, str):
            v = v.replace("'", "")
            val_str = f"'{v}'"
        elif isinstance(v, datetime):
            val_str = f"'{v.isoformat()}'"  # format datetime as ISO string
        else:
            val_str = str(v)
        parts.append(f"{key_str}: {val_str}")
    return "{" + ", ".join(parts) + "}"

### Solo ejecutar una vez. Las subsiguientes veces no será necesario

In [ ]:
#G.delete_all()

In [ ]:
for node_id, attrs in G_nx.nodes(data=True):
    node_id = re.sub(r'[^a-zA-Z0-9]', '_', node_id)
    sub_type = attrs.get("sub_type")
    label = re.sub(r'[^a-zA-Z0-9]', '_', attrs.get("label"))
    attrs["id"] = node_id
    attrs.pop('sub_type', None)
    attrs.pop('type', None)
    formatted = format_dict(attrs)
    query = "CREATE (n:{} {})".format(sub_type, formatted)
    try:
        driver.execute_query(query)
    except Exception as e:
        print(query)

In [ ]:
for source, target, attrs in G_nx.edges(data=True):
    source = re.sub(r'[^a-zA-Z0-9]', '_', source)
    target = re.sub(r'[^a-zA-Z0-9]', '_', target)
    if attrs.get("type") is None:
        relationship_type = "CONNECTED"
    else:
        relationship_type = attrs.get("type").upper()

    formatted_attrs = format_dict(attrs)
    query = f"""
        MATCH (a {{id: '{source}'}})
        MATCH (b {{id: '{target}'}})
        CREATE (a)-[r:{relationship_type} {formatted_attrs}]->(b)
    """
    try:
        driver.execute_query(query)
    except Exception as e:
        print(query)

#### Prueba

In [ ]:
data=driver.execute_query("""
                     MATCH (n1)-[r1:SENT]->(n2), 
                     (n2)-[r2:RECEIVED]->(n3)
                     WHERE n1.name='Mako'
                    RETURN n1, n2, n3, r1, r2;
                    """)

In [ ]:
records = []
for record in data.records:
    n1 = record.data()["n1"]
    n2 = record.data()["n2"]
    n3 = record.data()["n3"]
    r1 = record.data()["r1"]
    r2 = record.data()["r2"]

    records.append({
        'sender': n1['name'],
        'message_id': n2['id'],
        'content': n2['content'],
        'timestamp': n2['timestamp'],
        'receiver': n3['name'] if 'name' in n3 else n3['label'],
    })

df_Mako = pd.DataFrame(records)
df_Mako['timestamp'] = pd.to_datetime(df_Mako['timestamp'])  # Asegura que sea datetime

In [ ]:
df_Mako

In [ ]:
df_Mako['date'] = df_Mako['timestamp'].dt.date

hv_plot = hv.Curve(df_Mako.groupby('date').size().reset_index(name='count')).opts(
    xlabel='Fecha', ylabel='N° de Mensajes',
    title='Mensajes enviados por día',
    width=600, height=300, color='navy'
)
hv_plot

In [ ]:
hv_plot = hv.Bars(df_Mako['receiver'].value_counts().reset_index().rename(columns={'index':'Receptor', 'receiver':'Cantidad'})).opts(
    xlabel='Receptor', ylabel='Cantidad de mensajes',
    title='Destinatarios más frecuentes',
    width=600, height=300
)
hv_plot


In [ ]:
df_Mako['date'] = df_Mako['timestamp'].dt.date

hv_plot = hv.Curve(df_Mako.groupby('date').size().reset_index(name='count')).opts(
    xlabel='Fecha', ylabel='N° de Mensajes',
    title='Mensajes enviados por día',
    width=600, height=300, color='navy'
)
hv_plot


In [ ]:
Gmako = nx.DiGraph()
for i, row in df_Mako.iterrows():
    Gmako.add_edge(row['sender'], row['receiver'], label=row['message_id'])

hv_graph = hv.Graph.from_networkx(Gmako, nx.spring_layout).opts(
    node_color='lightblue',
    node_size=15,
    edge_color='gray',
    width=600,
    height=400,
    title="Red de comunicaciones desde Mako"
)
hv_graph

In [ ]:
df_Mako['hour'] = df_Mako['timestamp'].dt.hour
hv_hist = hv.Histogram(np.histogram(df_Mako['hour'], bins=24)).opts(
    xlabel='Hora del día', ylabel='Mensajes',
    title='Distribución horaria de mensajes',
    width=600, height=300
)
hv_hist


# Pregunta 1 - Variación temporal de las comunicaciones.

####   Clepper found that messages frequently came in at around the same time each day.

a.      Develop a graph-based visual analytics approach to identify any daily temporal patterns in communications.

b.      How do these patterns shift over the two weeks of observations?

c.       Focus on a specific entity and use this information to determine who has influence over them.

In [ ]:
data_comm=driver.execute_query("""
                     MATCH (n1)-[r1:SENT]->(n2), 
                     (n2)-[r2:RECEIVED]->(n3)
                    RETURN n1, n2, n3, r1, r2;
                    """)

In [ ]:
#data_comm

In [ ]:
records_comm = []
for record in data_comm.records:
    n1 = record.data()["n1"]
    n2 = record.data()["n2"]
    n3 = record.data()["n3"]
    r1 = record.data()["r1"]
    r2 = record.data()["r2"]

    records_comm.append({
        'sender': n1['name'],
        'message_id': n2['id'],
        'content': n2['content'],
        'timestamp': n2['timestamp'],
        'receiver': n3['name'] if 'name' in n3 else n3['label'],
    })

In [ ]:
df_comm = pd.DataFrame(records_comm)
df_comm['timestamp'] = pd.to_datetime(df_comm['timestamp'])  # Asegura que sea datetime

In [ ]:
df_chord = df_comm.groupby(['sender', 'receiver']).size().reset_index(name='weight')
df_chord.columns = ['source', 'target', 'weight']


In [ ]:
chord = hv.Chord(df_chord).opts(
    width=600, height=600,
    cmap='Category20', node_color='index',
    edge_cmap='Viridis', edge_color='weight',
    labels='name'
)
chord

In [ ]:
df_comm['day'] = df_comm['timestamp'].dt.date
df_comm['hour'] = df_comm['timestamp'].dt.hour


In [ ]:
grouped = df_comm.groupby(['sender', 'day', 'hour']).size().reset_index(name='count')


In [ ]:
grouped['datetime'] = pd.to_datetime(grouped['day'].astype(str)) + pd.to_timedelta(grouped['hour'], unit='h')


In [ ]:
heatmap = hv.HeatMap(grouped, kdims=['datetime', 'sender'], vdims='count').opts(
    width=900, height=600, cmap='Viridis', tools=['hover'],
    xrotation=45, xlabel='Fecha-Hora', ylabel='Sender'
)
heatmap

In [ ]:
# Elegimos un sender (por ejemplo, el primero en la lista)
sender_to_plot = grouped['sender'].unique()[0]

filtered = grouped[grouped['sender'] == sender_to_plot]

# Creamos tabla con días como filas y horas como columnas
pivot = filtered.pivot_table(index='day', columns='hour', values='count', fill_value=0)

plt.figure(figsize=(14, 6))
sns.heatmap(pivot, cmap='viridis', linewidths=0.5)
plt.title(f'Frecuencia de mensajes por hora por día – {sender_to_plot}')
plt.xlabel('Hora del día')
plt.ylabel('Fecha')
plt.tight_layout()
plt.show()


In [ ]:
df_stream = df_comm.groupby(['day', 'sender']).size().reset_index(name='count')

In [ ]:
df_stream_2 = df_comm.groupby(['day', 'hour','sender']).size().reset_index(name='count')

In [ ]:
df_comm

In [ ]:
timeframe = df_comm.groupby(pd.Grouper(key='timestamp', freq='1h')).count() #Group by hour and sum values
#["timestamp"].dt.time
timeframe

In [ ]:
timeframe_total = df_comm.groupby(pd.Grouper(key='timestamp', freq='1h')).size().reset_index(name='total_msgs')

# Crear gráfico de líneas
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=timeframe_total['timestamp'],
    y=timeframe_total['total_msgs'],
    mode='lines',
    name='Mensajes totales',
    line=dict(color='blue'),
    hovertemplate='Hora: %{x}<br>Mensajes: %{y}<extra></extra>'
))

fig.update_layout(
    title='Cantidad total de mensajes por hora',
    xaxis_title='Hora',
    yaxis_title='Mensajes',
    height=500,
    width=1000
)

fig.show()

In [ ]:
pivot_df = df_stream.pivot(index='day', columns='sender', values='count').fillna(0)

# Ordenamos por volumen total
sender_totals = pivot_df.sum().sort_values(ascending=False)

# Widget para elegir cuántos senders mostrar
num_senders_widget = widgets.IntSlider(
    value=10,
    min=2,
    max=min(30, len(sender_totals)),
    step=2,
    description='Top Senders:',
    continuous_update=False
)

# Función para actualizar el gráfico
def update_plot(num_senders):
    top_senders = sender_totals.head(num_senders).index
    data = []
    for sender in top_senders:
        data.append(go.Scatter(
            x=pivot_df.index,
            y=pivot_df[sender],
            mode='lines',
            stackgroup='one',
            name=sender,
            hovertemplate=f'Sender: {sender}<br>Día: %{{x}}<br>Mensajes: %{{y}}<extra></extra>'
        ))

    fig = go.Figure(data=data)
    fig.update_layout(
        title='Streamgraph de interacciones por día (Top Senders)',
        xaxis_title='Día',
        yaxis_title='Cantidad de mensajes',
        height=500,
        width=1000,
        legend_title_text='Sender',
    )
    fig.show()

# Conectar el widget
widgets.interact(update_plot, num_senders=num_senders_widget)


In [ ]:
df_stream

In [ ]:
num_senders_widget = widgets.IntSlider(
    value=10,
    min=2,
    max=30,
    step=2,
    description='Top Senders:',
    continuous_update=False
)

# Widget: hora del día
hour_widget = widgets.IntSlider(
    value=8,
    min=0,
    max=23,
    step=1,
    description='Hora:',
    continuous_update=False
)

# Función para actualizar el gráfico
def update_plot(num_senders, hour):
    # Filtramos por la hora seleccionada
    df_hour = df_stream_2[df_stream_2['hour'] == hour]

    # Pivot y top senders
    pivot_df = df_hour.pivot(index='day', columns='sender', values='count').fillna(0)
    sender_totals = pivot_df.sum().sort_values(ascending=False)
    top_senders = sender_totals.head(num_senders).index

    # Construcción del gráfico
    data = []
    for sender in top_senders:
        data.append(go.Scatter(
            x=pivot_df.index,
            y=pivot_df[sender],
            mode='lines',
            stackgroup='one',
            name=sender,
            hovertemplate=f'Sender: {sender}<br>Día: %{{x}}<br>Mensajes: %{{y}}<extra></extra>'
        ))

    fig = go.Figure(data=data)
    fig.update_layout(
        title=f'Streamgraph de interacciones por día — {hour}:00 hs (Top {num_senders} Senders)',
        xaxis_title='Día',
        yaxis_title='Cantidad de mensajes',
        height=500,
        width=1000,
        legend_title_text='Sender',
    )
    fig.show()

# Mostrar los widgets y conectar
widgets.interact(update_plot, num_senders=num_senders_widget, hour=hour_widget)

In [ ]:
# Asegurarse de que 'timestamp' esté en formato datetime
df_comm['timestamp'] = pd.to_datetime(df_comm['timestamp'])

# Agrupar por hora y contar mensajes por sender y receiver
grouped_sender = df_comm.groupby([
    pd.Grouper(key='timestamp', freq='1h'), 'sender'
]).size().reset_index(name='count_sender')

grouped_receiver = df_comm.groupby([
    pd.Grouper(key='timestamp', freq='1h'), 'receiver'
]).size().reset_index(name='count_receiver')

# Pivot para formato ancho (una columna por entidad)
pivot_sender = grouped_sender.pivot(index='timestamp', columns='sender', values='count_sender').fillna(0)
pivot_receiver = grouped_receiver.pivot(index='timestamp', columns='receiver', values='count_receiver').fillna(0)

# Widgets
max_senders = len(pivot_sender.columns)
max_receivers = len(pivot_receiver.columns)
max_entities = max(max_senders, max_receivers)

num_entities_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=max_entities,
    step=1,
    description='Top entidades:',
    continuous_update=False
)

sender_receiver_toggle = widgets.ToggleButtons(
    options=['Sender', 'Receiver'],
    description='Ver:',
    disabled=False,
    button_style=''
)

# Función para actualizar el gráfico
def update_plot(view_option, num_entities):
    if view_option == 'Sender':
        data_df = pivot_sender
        top_entities = data_df.sum().sort_values(ascending=False).head(num_entities).index
        title = "Mensajes por hora (Sender)"
    else:
        data_df = pivot_receiver
        top_entities = data_df.sum().sort_values(ascending=False).head(num_entities).index
        title = "Mensajes por hora (Receiver)"

    fig = go.Figure()
    for entity in top_entities:
        fig.add_trace(go.Scatter(
            x=data_df.index,
            y=data_df[entity],
            mode='lines',
            name=entity,
            hovertemplate=f'{entity}<br>Hora: %{{x}}<br>Mensajes: %{{y}}<extra></extra>'
        ))

    fig.update_layout(
        title=title,
        xaxis_title="Hora",
        yaxis_title="Cantidad de mensajes",
        height=500,
        width=1000,
        legend_title_text="Entidad"
    )
    fig.show()

# Conectar widgets
widgets.interact(update_plot,
                 view_option=sender_receiver_toggle,
                 num_entities=num_entities_slider)




# Pregunta 3. Pseudónimos

3 – It was noted by Clepper's intern that some people and vessels are using pseudonyms to communicate.

a.      Expanding upon your prior visual analytics, determine who is using pseudonyms to communicate, and what these pseudonyms are.

·         Some that Clepper has already identified include: “Boss”, and “The Lookout”, but there appear to be many more.

·         To complicate the matter, pseudonyms may be used by multiple people or vessels.

b.      Describe how your visualizations make it easier for Clepper to identify common entities in the knowledge graph.

c.       How does your understanding of activities change given your understanding of pseudonyms?

In [ ]:
# Contar cuántas veces cada entidad fue sender y receiver
sender_counts = df_comm['sender'].value_counts().rename('sent_count')
receiver_counts = df_comm['receiver'].value_counts().rename('received_count')

# Combinar en un solo DataFrame
activity_df = pd.concat([sender_counts, receiver_counts], axis=1).fillna(0).astype(int)


activity_df['diff'] = (activity_df['sent_count'] - activity_df['received_count']).abs()
activity_df['total'] = activity_df['sent_count'] + activity_df['received_count']
activity_df = activity_df.sort_values(by='total', ascending=False)

In [ ]:
activity_df

In [ ]:
activity_df

# Eje x y posiciones
x = np.arange(len(activity_df))
width = 0.3

# Crear gráfico
fig, ax = plt.subplots(figsize=(18, 6))

bars1 = ax.bar(x - width/2, activity_df['sent_count'], width, label='Enviados')
bars2 = ax.bar(x + width/2, activity_df['received_count'], width, label='Recibidos')

# Nombres reales de las entidades como etiquetas
ax.set_xticks(x)
ax.set_xticklabels(activity_df.index, rotation=70, ha='right')

# Estética
ax.set_xlabel('Entidad')
ax.set_ylabel('Cantidad de mensajes')
ax.set_title('Mensajes enviados y recibidos por entidad')
ax.legend()
ax.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()
#